# 异常抛出

在路由控制中我们定义了对用户的怎删改查功能,但如果我们要对异常做定制要咋办呢?

flask提供了重新定向和错误功能

Api|说明
---|---
redirect()|重新定向
abort()|放弃请求并返回错误代码

如果要自定义错误输出,可以使用装饰器`render_template`来实现

In [18]:
%%writefile codes/c3/app.py
# --*-- coding:utf-8 --*--
from __future__ import print_function, unicode_literals, division

from flask import Flask,jsonify,request,abort
from flask.views import MethodView
import json
app = Flask(__name__)

Userlist = []
def inc(cls):
    counter = {"result":0}
    def wrap(*args):
        counter["result"]+=1
        result = cls(counter["result"],*args)
        global Userlist
        Userlist.append(result)
        return result
    return wrap
@inc
class User(object):
    def __str__(self):
        return "<USER: id-{self._id}-{self.name}>".format(self=self)
    def __repr__(self):
        return self.__str__()
    def __init__(self,_id,name,password,type="local"):
        self._id=_id
        self.name = name
        self.password = password
        self.type=type
        
        
        
@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404

class UsersAPI(MethodView):
    
    def get(self):
        return jsonify({"result":map(lambda x:x.__str__(),Userlist)})
        
    def post(self):
        data = json.loads(request.data)
        user = User(data.get("name"),data.get("password"))
        return jsonify({"result":"save {name} done!".format(name=user.name)})
    
class UserAPI(MethodView):
    
    def get(self,user_id):
        try:
            user = filter(lambda x:True if x._id==user_id else False,Userlist)[0]
        except IndexError as e:
            abort(404)
            
        result = {"name":user.name,"password":user.password,"type":user.type}
        return jsonify({"result":result})
        
    def put(self,user_id):
        data = request.args
        user = filter(lambda x: True if x._id==user_id else False,Userlist)[0]
        if request.args.get("name"):
            user.name=data.get("name")
        if request.args.get("password"):
            user.password=data.get("password")
        
        return jsonify({"result":"update {name} done!".format(name=user_id)})    
    def delete(self,user_id):
        global Userlist
        Userlist = filter(lambda x: False if x._id==user_id else True,Userlist)
        return jsonify({"result":"delete {id} done!".format(id=user_id)})   
        

app.add_url_rule('/users/',view_func=UsersAPI.as_view('users'.encode("utf-8")))
app.add_url_rule('/user/<int:user_id>',view_func=UserAPI.as_view('user'.encode("utf-8")))

if __name__ == '__main__':
    app.run(debug=True)

Overwriting codes/c3/app.py


In [19]:
import requests
import json

In [20]:
requests.post("http://127.0.0.1:5000/users/",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'result': u'save niuniu done!'}

In [21]:
requests.get("http://127.0.0.1:5000/users/").json()

{u'result': [u'<USER: id-1-niuniu>']}

In [22]:
requests.get("http://127.0.0.1:5000/user/2").json()

{u'code': 404, u'message': u'404 not found'}